In [1]:
%matplotlib inline

In [20]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from vtk_rw import read_vtk
import seaborn as sns
from scipy.optimize import curve_fit
import sklearn

In [3]:
def load_pickle(pkl_file):
    pkl_in = open(pkl_file, 'r')
    pkl_dict = pickle.load(pkl_in)
    pkl_in.close()
    return pkl_dict

In [7]:
infl=150
lh_mesh_file = '/scr/ilz3/myelinconnect/new_groupavg/surfs/lowres/inflated/lh_lowres_new_taubin_%i.vtk'%infl
lh_sulc_file = '/scr/ilz3/myelinconnect/new_groupavg/surfs/lowres/inflated/lh_lowres_new_taubin_%i_sulc.npy'%infl
rh_mesh_file = '/scr/ilz3/myelinconnect/new_groupavg/surfs/lowres/inflated/rh_lowres_new_taubin_%i.vtk'%infl
rh_sulc_file = '/scr/ilz3/myelinconnect/new_groupavg/surfs/lowres/inflated/rh_lowres_new_taubin_%i_sulc.npy'%infl
fullmask_file = '/scr/ilz3/myelinconnect/new_groupavg/masks/fullmask_lh_rh_new.npy'
lh_sulc = np.load(lh_sulc_file)
lv, lf, _ = read_vtk(lh_mesh_file)
rh_sulc = np.load(rh_sulc_file)
rv, rf, _ = read_vtk(rh_mesh_file)
fullmask = np.load(fullmask_file)

### Load and prep embeddings

In [8]:
embed_file='/scr/ilz3/myelinconnect/new_groupavg/embed/both_smooth_3_embed.npy'
embed_viz_file='/scr/ilz3/myelinconnect/new_groupavg/embed/both_smooth_3_embed_viz.npy'
dict_file='/scr/ilz3/myelinconnect/new_groupavg/embed/both_smooth_3_embed_dict.pkl'
dict_viz_file='/scr/ilz3/myelinconnect/new_groupavg/embed/both_smooth_3_embed_dict_viz.pkl'

embed_dict = load_pickle(dict_file)
eigenval=embed_dict['lambdas']
perc_var = eigenval/np.sum(eigenval)*100
# normalize vectors
embed_masked = np.zeros((embed_dict['vectors'].shape[0], embed_dict['vectors'].shape[1]-1))
for comp in range(100):
    embed_masked[:,comp]=(embed_dict['vectors'][:,comp+1]/embed_dict['vectors'][:,0])

# unmask the embedding, that has been saved in masked form
idcs=np.arange(0,(lv.shape[0]+rv.shape[0]))
nonmask=np.delete(idcs, fullmask)
embed = np.zeros(((lv.shape[0]+rv.shape[0]),100))
embed[nonmask] = embed_masked

### Load models

In [9]:
t1_predict_file_0 = '/scr/ilz3/myelinconnect/new_groupavg/model/linear_combination/t1avg/smooth_1.5/both_t1avg_by_fc_maps_0.pkl'
t1_predict_file_best = '/scr/ilz3/myelinconnect/new_groupavg/model/linear_combination/t1avg/smooth_1.5/both_t1avg_by_fc_maps_best.pkl'
t1_predict_0 = load_pickle(t1_predict_file_0)
t1_predict_best = load_pickle(t1_predict_file_best)

In [12]:
scatter_mask = np.where(t1_predict_0['t1']>1500)
FC1 = -embed[:,0][scatter_mask]
T1 = t1_predict_0['t1'][scatter_mask]

xdata = FC1
ydata = (T1-np.mean(T1))/np.std(T1)
x = np.linspace(-2,2,100) #xdata.min(), xdata.max(), 100)

In [13]:
FCbest=t1_predict_best['modelled_fit'][scatter_mask]
xbest = np.linspace(1850,2150,100)

### Calculate p values with sandwich estimator

In [14]:
import scipy.stats as stats
print "Pearson FC1:", stats.pearsonr(embed[:,0][scatter_mask],t1_predict_0['t1'][scatter_mask])[0]
print "Spearman FC1:", stats.spearmanr(embed[:,0][scatter_mask],t1_predict_0['t1'][scatter_mask])[0]
print ''
print "Pearson FC1,5,6:",stats.pearsonr(t1_predict_best['modelled_fit'][scatter_mask],t1_predict_0['t1'][scatter_mask])[0]
print "Spearman FC1,5,6:", stats.spearmanr(t1_predict_best['modelled_fit'][scatter_mask],t1_predict_0['t1'][scatter_mask])[0]


Pearson FC1: -0.528208342431
Spearman FC1: -0.546362242198

Pearson FC1,5,6: 0.667511548784
Spearman FC1,5,6: 0.705016286249


In [15]:
import statsmodels.api as sm
import statsmodels.stats as sms
#X = np.column_stack((np.ones(FC1.shape), FC1))
X = np.column_stack((np.ones(FC1.shape), FC1, -embed[:,4][scatter_mask], -embed[:,5][scatter_mask]))
model = sm.OLS(T1, X)
results = model.fit()

In [16]:
print 'classic covariance matrix'
print results.cov_params()
print ''
print 'Sandwich robust covariance matrix'
sandwich=sms.sandwich_covariance.cov_white_simple(results)
print sandwich

classic covariance matrix
[[  8.31044671e-02  -1.01266055e-03  -2.55333647e-04  -1.75812456e-04]
 [ -1.01266055e-03   8.32009562e-02  -5.72317391e-05  -3.59824890e-04]
 [ -2.55333647e-04  -5.72317391e-05   8.30262280e-02  -4.69815842e-04]
 [ -1.75812456e-04  -3.59824890e-04  -4.69815842e-04   8.30912946e-02]]

Sandwich robust covariance matrix
[[ 0.08314991 -0.00267364  0.00067727 -0.00269965]
 [-0.00267364  0.09626431 -0.01662708 -0.00412796]
 [ 0.00067727 -0.01662708  0.08385298 -0.01948418]
 [-0.00269965 -0.00412796 -0.01948418  0.09438307]]


In [17]:
print 'Classic standard errors'
print results.bse[0], results.bse[1], results.bse[2], results.bse[3]
print ''
print 'Sandwich standard errors'
print np.sqrt(sandwich[0][0]), np.sqrt(sandwich[1][1]), np.sqrt(sandwich[2][2]), np.sqrt(sandwich[3][3])

Classic standard errors
0.288278454051 0.288445759569 0.288142721565 0.288255606312

Sandwich standard errors
0.288357253422 0.310264895007 0.289573783041 0.307218273311


In [18]:
print 'Classic confidence intervals'
print [results.params[0]-1.96*results.bse[0], results.params[0]+1.96*results.bse[0]]
print [results.params[1]-1.96*results.bse[1], results.params[1]+1.96*results.bse[1]]
print [results.params[2]-1.96*results.bse[2], results.params[2]+1.96*results.bse[2]]
print [results.params[3]-1.96*results.bse[3], results.params[3]+1.96*results.bse[3]]
print ''
print 'Sandwich robust confidence intervals'
print [results.params[0]-1.96*sandwich[0][0], results.params[0]+1.96*sandwich[0][0]]
print [results.params[1]-1.96*sandwich[1][1], results.params[1]+1.96*sandwich[1][1]]
print [results.params[2]-1.96*sandwich[2][2], results.params[2]+1.96*sandwich[2][2]]
print [results.params[3]-1.96*sandwich[3][3], results.params[3]+1.96*sandwich[3][3]]

Classic confidence intervals
[1983.6877226741487, 1984.8177742140294]
[70.273112184426211, 71.403819561937681]
[34.563933863026868, 35.693453331561791]
[41.359776948746969, 42.489738925489632]

Sandwich robust confidence intervals
[1984.0897746291112, 1984.415722259067]
[70.649787835237035, 71.027143911126856]
[34.964341764677783, 35.293045429910876]
[41.73976712490385, 42.109748749332752]


In [19]:
print 'Beta 0'
print 'classic p', stats.norm.sf(abs(results.params[0]/results.bse[0]))*2
print 'HAC  p', stats.norm.sf(abs(results.params[0]/sandwich[0][0]))*2
print ''
print 'Beta 1'
print 'classic p', stats.norm.sf(abs(results.params[1]/results.bse[1]))*2
print 'HAC  p', stats.norm.sf(abs(results.params[1]/sandwich[1][1]))*2
print ''
print 'Beta 2'
print 'classic p', stats.norm.sf(abs(results.params[2]/results.bse[2]))*2
print 'HAC  p', stats.norm.sf(abs(results.params[2]/sandwich[2][2]))*2
print ''
print 'Beta 3'
print 'classic p', stats.norm.sf(abs(results.params[3]/results.bse[3]))*2
print 'HAC  p', stats.norm.sf(abs(results.params[3]/sandwich[3][3]))*2

Beta 0
classic p 0.0
HAC  p 0.0

Beta 1
classic p 0.0
HAC  p 0.0

Beta 2
classic p 0.0
HAC  p 0.0

Beta 3
classic p 0.0
HAC  p 0.0


To find the p-values we can first calculate the z-statistics (coefficients divided by their corresponding standard errors), and compare the squared z-statistics to a chi-squared distribution on one degree of freedom

### Test for heteroscedasticity

In [21]:
from statsmodels.compat import lzip
name = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.api.het_breushpagan(results.resid, results.model.exog)
lzip(name, test)

[('Lagrange multiplier statistic', 65.431817668059324),
 ('p-value', 4.0549674129473383e-14),
 ('f-value', 21.821725695864924),
 ('f p-value', 4.0234070978690622e-14)]

In [22]:
test_white = sms.api.het_white(results.resid, results.model.exog)
lzip(name, test_white)

[('Lagrange multiplier statistic', 4952.5853642667898),
 ('p-value', 0.0),
 ('f-value', 573.81400019885427),
 ('f p-value', 0.0)]

In [23]:
name = ['F statistic', 'p-value']
test_gold = sms.api.het_goldfeldquandt(results.resid, results.model.exog)
lzip(name, test_white)

[('F statistic', 4952.5853642667898), ('p-value', 0.0)]